# Demo KartAI

## Import pakker

In [382]:
import os
import fiona
import pyarrow.parquet as pq
import geopandas as gpd
import pandas as pd
import time as time_module
import glob
import datetime
import json
import re
import builtins
from shapely import wkt
from shapely.geometry import box
from IPython.display import display

## Konstante variabler

In [383]:
ABSOLUTE_PATH = os.getcwd()
ROT_FOLDER = "Data"
ROT_PATH = os.path.join(ABSOLUTE_PATH, ROT_FOLDER)
SUPPORTED_FORMATS = ['.parquet', '.geojson', '.json', '.shp', '.gpkg', '.csv', '.gml']
FOLDER = {
    'raw': os.path.join(ROT_PATH, 'raw'), 
    'processed': os.path.join(ROT_PATH, 'processed'), 
    'archive': os.path.join(ROT_PATH, 'archive'),
    'logs': os.path.join(ROT_PATH, 'logs')
}

## Diverse Sjekker

In [384]:
def is_geoparquet(file_path):
    metadata = pq.read_metadata(file_path)
    return "geo" in metadata.metadata if metadata.metadata else False

In [385]:
def create_target_file_path(source_file_path, target_folder):
    """Lager målfilsti basert på kildesti og evt. målmappe"""
    filename = os.path.basename(source_file_path)
    filename_Without_type = os.path.splitext(filename)[0]

    if target_folder is None:
        # Hvis ingen målmappe er angitt, bruk samme mappe med _geo.parquet
        dir_navn = os.path.dirname(source_file_path)
        return os.path.join(dir_navn, f"{filename_Without_type}_geo.parquet")
    else:
        # Hvis målmappe er angitt, plasser filen der
        return os.path.join(source_file_path, f"{filename_Without_type}.parquet")

In [386]:
def valid_conversion(filepath, filename, filetype):
    """Sjekker om en fil er gyldig for konvertering"""
    # Ikke konverter loggfiler
    if filename == "conversion_log" or filepath.endswith("conversion_log.json"):
        return False

    # Sjekk om filen har et støttet format
    if filetype not in SUPPORTED_FORMATS:
        return False

    # Parquet-filer trenger ekstra sjekk
    if filetype == '.parquet':
        # Ikke ta med filer som allerede er GeoParquet
        if is_geoparquet(filepath):
            return False
        # Ikke ta med filer som allerede har _geo suffix
        if filename.endswith('_geo'):
            return False

    return True

In [387]:
def check_file_convert(filepath, filename, convert_files, convert_logs):
    """Sjekker om en fil trenger konvertering basert på konverteringshistorikk"""
    # Sjekk loggfilen
    if filename in convert_logs:
        # Sjekk om den konverterte filen faktisk eksisterer
        target_path = convert_logs[filename].get('målsti', None)
        if target_path and os.path.exists(target_path):
            # Filen finnes - sjekk om den har blitt endret siden sist konvertering
            sist_endret_tid = os.path.getmtime(filepath)
            sist_konvertert_tid = convert_logs[filename].get('tidspunkt', 0)
            if sist_endret_tid <= sist_konvertert_tid:
                return False
        else:
            # Filen i loggfilen eksisterer ikke - den bør konverteres på nytt
            print(f"Konvertert fil {target_path} finnes ikke, konverterer på nytt...")
            return True

    # Sjekk om filen allerede finnes i processed-mappen
    return filename not in convert_files

## Get-funskjoner

In [388]:
def get_convert_files():
    """Henter ut alle allerede konverterte filer"""

    # Return an empty set if the folder does not exist
    if not os.path.exists(FOLDER['processed']):
        return None

    convert_files = set()
    # Iterate through files in the directory
    for root, _, filer in os.walk(FOLDER['processed']):
        for fil in filer:
            if fil.endswith('.parquet'):
                base_navn = os.path.splitext(fil)[0]
                convert_files.add(base_navn)

    return convert_files

In [389]:
def get_exists_geoparquet():
    """Finner eksisterende GeoParquet-filer"""
    geoparquet_files = []

    for root, _, files in os.walk(FOLDER['raw']):
        for file in files:
            if file.endswith('.parquet'):
                filepath = os.path.join(root, file)
                if is_geoparquet(filepath) and filepath not in files:
                    geoparquet_files.append(filepath)

    return geoparquet_files

In [390]:
def get_new_files():
    """Finner filer som ikke har blitt konvertert ennå"""
    convert_files = get_convert_files()
    #convert_logs = read_log()

    nye_filer = []

    for root, _, files in os.walk(FOLDER['raw']):
        for file in files:
            filepath = os.path.join(root, file)
            filetype = os.path.splitext(file)[1].lower()
            filename = os.path.splitext(file)[0]

            # Sjekk om filen er gyldig for konvertering
            if not valid_conversion(filepath, filename, filetype):
                continue

            # Sjekk om filen trenger konvertering
            #if check_file_convert(filepath, filename, convert_files):
            nye_filer.append(filepath)

    return nye_filer

### Konverting til parquet

#### sjekk om det er Parquet

In [391]:
def convert_latlon_to_gdf(df):
    """Convert a DataFrame with latitude/longitude columns to a GeoDataFrame."""
    if {'longitude', 'latitude'}.issubset(df.columns):
            df['geometry'] = gpd.points_from_xy(df['longitude'], df['latitude'])
            return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
    return None

In [392]:
def detect_wkt_geometry(df):
    """Detect a potential geometry column in WKT format and convert it to a GeoDataFrame."""
    pattern = re.compile(r'geom|coord|point|polygon|linestring|wkt', re.IGNORECASE)
    geo_col = [col for col in df.columns if pattern.search(col)]

    for col in geo_col:
        if df[col].dtype == 'object':
            # Prøv å konvertere fra WKT-format
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
    return None

In [393]:
def converter_parquet(path):
    """Konverterer parquet-fil til GeoDataFrame"""
    if isinstance(path, str) and os.path.exists(path):
        df = pd.read_parquet(path)
    
    # Try latitude/longitude conversion first
    gdf = convert_latlon_to_gdf(df)
    if gdf is not None:
        return gdf

    # Try detecting and converting WKT geometry
    gdf = detect_wkt_geometry(df)
    if gdf is not None:
        return gdf

    print("No valid geometry columns found.")
    return None
    

#### Sjekk om det er CSV

In [394]:
def converter_csv(filsti):
    """Konverterer CSV-fil til GeoDataFrame"""
    df = pd.read_csv(filsti)

    # Sjekk for vanlige lat/long kolonnenavn
    lat_kolonner = ['latitude', 'lat', 'y', 'breddegrad']
    lon_kolonner = ['longitude', 'long', 'lon', 'x', 'lengdegrad']

    lat_col = next((col for col in lat_kolonner if col in df.columns), None)
    lon_col = next((col for col in lon_kolonner if col in df.columns), None)

    if lat_col and lon_col:
        return gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
            crs="EPSG:4326"
        )

    return None

#### Sjekk om det er gml

In [395]:
def convert_gml(path):
    layers = fiona.listlayers(path)
    if layers:
        print(f"GML-fil har {len(layers)} lag: {', '.join(layers)}")
        # Les det første laget, som er standard
        return gpd.read_file(path, layer=layers[0])
    else:
        return gpd.read_file(path)

#### Konvertere filer til geoparquet

In [396]:
def convert_filetype(path):
    """Konverterer fil til GeoDataFrame basert på filendelse"""
    
    filetype = os.path.splitext(path)[1].lower()
    if filetype == '.parquet':
        return converter_parquet(path)

    elif filetype in ['.geojson', '.json', '.shp', '.gpkg']:
        return gpd.read_file(path)

    elif filetype == '.gml':
        return convert_gml(path)
    
    elif filetype == '.csv':
        return converter_csv(path)

    print(f"Klarte ikke å finne match konverting til filendelse {filetype}")
    return None

In [397]:
def converter_to_geoparquet(source_filepath, target_folder_path=None):
    """Konverterer ulike geografiske formater til GeoParquet-format"""
    
    target_file_path = create_target_file_path(source_filepath, target_folder_path)
    
    # Sørg for at målmappen eksisterer
    os.makedirs(os.path.dirname(source_filepath), exist_ok=True)
    
    # Konverter filen basert på filtype
    gdf = convert_filetype(source_filepath)

    # Hvis vi har en gyldig geodataframe, lagre som GeoParquet
    if gdf is not None:
        gdf.to_parquet(target_file_path)
        return target_file_path

    return False

### Logging

In [398]:
def read_log():
    """Leser inn eksisterende konverteringslogg"""
    if not os.path.exists(FOLDER['logs']):
        return {}
    with builtins.open(FOLDER['logs'], 'r') as f:
        return json.load(f)

In [399]:
def generate_logs(file, target_file_path):
    """Lager et nytt logginnslag for en konvertert fil"""
    base_name = os.path.splitext(os.path.basename(fil))[0]
    today = datetime.datetime.now()

    return {
        base_name: {
            'kildesti': file,
            'målsti': target_file_path,
            'tidspunkt': today.timestamp(),
            'dato': today.strftime('%Y-%m-%d %H:%M:%S')
        }
    }

In [400]:
def save_log(logdata):
    """Lagrer konverteringslogg til fil"""
    with builtins.open(FOLDER['logs'], 'w') as f:
        json.dump(logdata, f, indent=2)

In [401]:
def update_logs(file, target_file_path):
    """Oppdaterer konverteringsloggen med ny filinfo"""
    # Last inn eksisterende logg
    convert_logger = read_log()

    # Lag og legg til nytt logginnslag
    new_log = generate_logs(file, target_file_path)
    convert_logger.update(new_log)

    # Lagre oppdatert logg
    save_log(convert_logger)

## Setup

In [402]:
def setup():
    # Sjekk om mappen finnes, hvis ikke - lag den
    if not os.path.exists(ROT_FOLDER):
        os.makedirs(ROT_FOLDER, exist_ok=True)
        print(f"Opprettet rotmappe: {ROT_FOLDER}")
    
        # Opprett undermappe-struktur
        print(f"Creating folder in {ROT_FOLDER}")
        for folderName in FOLDER:
            os.makedirs(FOLDER[folderName], exist_ok=True)
            print(f"Opprettet mappe: {FOLDER[folderName]}")
    else:
        print(f"Mappen {ROT_FOLDER} finnes allerede")

## Hovedfunskjoner

In [403]:
def konverter_filer(new_files):
    """Konverterer filer og oppdaterer logg"""
    resultater = {
        "konvertert": [],
        "feilet": []
    }

    for filepath in new_files:
        # Konverter filen
        target_filepath = converter_parquet(filepath)

        if target_filepath:
            resultater["konvertert"].append(filepath)
            update_logs(FOLDER['log'], filepath)
        else:
            resultater["feilet"].append(filepath)

    return resultater

In [404]:
def skriv_oppsummering(resultater):
    """Skriver oppsummering av konverteringsprosessen"""
    print("\n")
    print("OPPSUMMERING")
    print(f"Katalog: {ROT_PATH}")
    print(f"Mappestruktur: Raw={FOLDER['raw']}, Processed={FOLDER['processed']}")

    totalt_antall = len(resultater["alle_filer"])
    antall_allerede_geo = len(resultater["allerede_geoparquet"])
    antall_konvertert = len(resultater["konvertert"])
    antall_feilet = len(resultater["feilet"])

    print(f"Totalt antall geografiske filer funnet: {totalt_antall}")
    print(f"Antall parquet-filer som allerede var GeoParquet: {antall_allerede_geo}")
    print(f"Antall filer konvertert til GeoParquet: {antall_konvertert}")
    print(f"Antall filer som ikke kunne konverteres: {antall_feilet}")

    if resultater["konvertert"]:
        print("\nKonverterte filer:")
        for fil in resultater["konvertert"]:
            print(f"- {fil}")

    if resultater["feilet"]:
        print("\nFiler som ikke kunne konverteres:")
        for fil in resultater["feilet"]:
            print(f"- {fil}")

In [405]:
def behandle_alle_filer_i_mappe():
    """Behandler alle geografiske filer i angitt mappe og konverterer til GeoParquet."""
    # Validering og mappestruktur (gjør dette først!)
    if not os.path.exists(ROT_PATH):
        print(f"Vennligst oppgi en gyldig mappe som du har tilgang til.")
        print(f"For eksempel: './data' (relativ sti) eller '~/data' (i hjemmekatalogen)")
        return None

    # Uansett om opprett_struktur er True eller False, sørg for at processed-mappen eksisterer
    processed_sti = FOLDER['processed']
    if not os.path.exists(processed_sti):
        try:
            os.makedirs(processed_sti, exist_ok=True)
            print(f"Opprettet processed-mappe: {processed_sti}")
        except Exception as e:
            print(f"FEIL: Kunne ikke opprette processed-mappen: {e}")

    # Bruk processed-sti som målmappe i stedet for prosessert_mappesti fra oppsett_mappestier
    if not os.path.exists(processed_sti):
        try:
            os.makedirs(processed_sti, exist_ok=True)
        except Exception as e:
            print(f"FEIL: Kunne ikke opprette målmappen '{processed_sti}': {e}")
            return None

    # Finn nye filer som trenger konvertering
    nye_filer = get_new_files()

    # Konverter filene og samle resultatene - bruk processed_sti som målmappe
    konverteringsresultater = konverter_filer(
        nye_filer
    )

    # Finn eksisterende GeoParquet-filer
    allerede_geoparquet = get_exists_geoparquet()

    # Sammenstill resultater
    resultater = {
        "alle_filer": nye_filer,
        "konvertert": konverteringsresultater["konvertert"],
        "feilet": konverteringsresultater["feilet"],
        "allerede_geoparquet": allerede_geoparquet
    }
    skriv_oppsummering(resultater)

    return resultater

In [406]:
# Sjekk at mappen eksisterer før vi fortsetter
setup()

# Kjører konverteringsprosessen
resultater = behandle_alle_filer_i_mappe()

# Viser resultatet
if resultater:
    print(f"\nTotalt konverterte filer: {len(resultater['konvertert'])}")
    print(f"Filer som allerede var i GeoParquet-format: {len(resultater['allerede_geoparquet'])}")
else:
    print("Konverteringen feilet. Sjekk filtype og format!")

Mappen Data finnes allerede


ValueError: The truth value of a GeoDataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Filtrering

In [331]:
# Les geoparquet-filen
fil_sti = "data/raw/hais_2025-01-01.snappy.parquet"
gdf = gpd.read_parquet(fil_sti)

# Filtrering basert på attributt/kolonne
if 'date_time_utc' in gdf.columns:
    filtrert_på_dato = gdf[gdf['date_time_utc'] >= '2025-01-01 16:00:00']
    print(f"Antall rader etter datofiltrering: {len(filtrert_på_dato)}")
    display(filtrert_på_dato.head())

# Filtrering basert på geometri (f.eks. et område)
område = box(5.0, 60.0, 11.0, 60.0)
innenfor_område = gdf[gdf.geometry.intersects(område)]
print(f"Antall rader innenfor det definerte området: {len(innenfor_område)}")
display(innenfor_område.head())

# Kombinert filtrering med flere kriterier
if 'attributt' in gdf.columns:
    kombinert_filter = gdf[(gdf.geometry.intersects(område)) & (gdf['attributt'] > 10)]
    print(f"Antall rader etter kombinert filtrering: {len(kombinert_filter)}")
    display(kombinert_filter.head())

FileNotFoundError: [WinError 2] Failed to open local file 'data/raw/hais_2025-01-01.snappy.parquet'. Detail: [Windows error 2] Systemet finner ikke angitt fil.


## Partisjonerte

In [31]:
# 1. Opprett en mappe for partisjonerte data
output_mappe = "Data/partitioned_hour"
os.makedirs(output_mappe, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
data_for_partisjonering = innenfor_område.copy()

# 3. Bruk .loc for å legge til time-kolonnen
data_for_partisjonering.loc[:, 'time'] = data_for_partisjonering['date_time_utc'].dt.hour

# 4. Partisjonering basert på time og ship_type
for time, gruppe_time in data_for_partisjonering.groupby('time'):
    time_mappe = os.path.join(output_mappe, f"time={time:02d}")
    os.makedirs(time_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_time.groupby('ship_type'):
        skip_mappe = os.path.join(time_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('time', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

NameError: name 'innenfor_område' is not defined

In [33]:
partisjon_mappe = "Data/partitioned_hour"

# Hente data for en spesifikk måned
ønsket_måned = "2025-01"
måned_sti = os.path.join(partisjon_mappe, f"år_måned={ønsket_måned}")

if os.path.exists(måned_sti):
    print(f"Leser data for {ønsket_måned}...")
    # Finn alle geoparquet-filer i denne månedens mappe (inkludert undermapper)
    filer = glob.glob(os.path.join(måned_sti, "**/*.parquet"), recursive=True)
    # Les og kombiner alle filene
    dataframes = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        dataframes.append(gdf)
    
    if dataframes:
        månedsdata = pd.concat(dataframes)
        print(f"Hentet {len(månedsdata)} rader for {ønsket_måned}")
        display(månedsdata.head())
    else:
        print(f"Ingen data funnet for {ønsket_måned}")
else:
    print(f"Ingen mappe funnet for {ønsket_måned}")

# Hente data for en spesifikk skipstype
ønsket_skipstype = 30
skipstype_stier = glob.glob(os.path.join(partisjon_mappe, f"**/ship_type={ønsket_skipstype}/*.parquet"), recursive=True)

if skipstype_stier:
    print(f"\nLeser data for skipstype {ønsket_skipstype}...")
    skipstype_dataframes = []
    for fil in skipstype_stier:
        gdf = gpd.read_parquet(fil)
        skipstype_dataframes.append(gdf)
    
    skipstype_data = pd.concat(skipstype_dataframes)
    print(f"Hentet {len(skipstype_data)} rader for skipstype {ønsket_skipstype}")
    display(skipstype_data.head())
else:
    print(f"Ingen data funnet for skipstype {ønsket_skipstype}")

Ingen mappe funnet for 2025-01
Ingen data funnet for skipstype 30


In [34]:
# 1. Opprett en mappe for partisjonerte data
output_mappe = "Data/partitioned_minutes"
os.makedirs(output_mappe, exist_ok=True)

# 2. Lag en kopi for å unngå SettingWithCopyWarning
data_for_partisjonering = innenfor_område.copy()

# 3. Bruk .loc for å legge til 10-minutters intervall kolonne
# Dette gir intervaller 0-143 for hele dagen (144 intervaller på 10 minutter)
data_for_partisjonering.loc[:, 'minuttgruppe'] = (
    data_for_partisjonering['date_time_utc'].dt.hour * 6 + 
    data_for_partisjonering['date_time_utc'].dt.minute // 10
)

# 4. Partisjonering basert på minuttgruppe og ship_type
for minuttgruppe, gruppe_minutt in data_for_partisjonering.groupby('minuttgruppe'):
    # Konverter minuttgruppe til time og minutt for mappe-strukturen
    time = minuttgruppe // 6
    minutt = (minuttgruppe % 6) * 10
    
    # Lag en lesbar mappestruktur (time_minutt=HH_MM)
    minutt_mappe = os.path.join(output_mappe, f"time_minutt={time:02d}_{minutt:02d}")
    os.makedirs(minutt_mappe, exist_ok=True)
    
    for ship_type, gruppe_final in gruppe_minutt.groupby('ship_type'):
        skip_mappe = os.path.join(minutt_mappe, f"ship_type={ship_type}")
        os.makedirs(skip_mappe, exist_ok=True)
        
        # Bruk en kopi av dataene
        data_å_lagre = gruppe_final.copy()
        
        # Fjern partisjoneringskolonnen før lagring
        fil_sti = os.path.join(skip_mappe, f"data.parquet")
        data_å_lagre.drop('minuttgruppe', axis=1).to_parquet(fil_sti)
        
        print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")

NameError: name 'innenfor_område' is not defined

## Strømming

In [35]:
# Simulert datastrømming over timer
partisjon_mappe = "Data/partitioned_hour"

# Finn alle time-mapper sortert kronologisk
alle_timer = sorted([d for d in os.listdir(partisjon_mappe) if d.startswith("time=")])

print("\nSimulerer datastrømming over timer...")
for time_dir in alle_timer:
    time_verdi = time_dir.split("=")[1]
    time_sti = os.path.join(partisjon_mappe, time_dir)
    
    print(f"\nProsesserer data for time {time_verdi}...")
    
    # Finn alle geoparquet-filer for denne timen
    filer = glob.glob(os.path.join(time_sti, "**/*.parquet"), recursive=True)
    
    # Les og bearbeid hver fil
    time_data = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        time_data.append(gdf)
        
    if time_data:
        samlet_data = pd.concat(time_data)
        print(f"Time {time_verdi}: Lastet {len(samlet_data)} rader fra {len(filer)} filer")
        
        # Her kan du gjøre din analyse for denne timen
        # For eksempel:
        print(f"  Gjennomsnittlig hastighet (SOG): {samlet_data['speed_over_ground'].mean():.2f}")
        print(f"  Antall unike skip: {samlet_data['mmsi'].nunique()}")
        
        # For å visualisere bevegelse over tid, kan du sortere på tidspunkt innen timen
        samlet_data_sortert = samlet_data.sort_values('date_time_utc')
        
        # (Her kunne du lagt til kode for visualisering)
        
    else:
        print(f"Ingen data funnet for time {time_verdi}")
    
    # Simuler tid mellom timer
    time_module.sleep(1)  # Vent ett sekund mellom hver time

print("\nDatastrømming komplett!")


Simulerer datastrømming over timer...

Datastrømming komplett!


In [36]:
# Simulert datastrømming over 10-minutters intervaller
partisjon_mappe = "Data/partitioned_minutes"

# Finn alle minutt-mapper sortert kronologisk
alle_intervaller = sorted([d for d in os.listdir(partisjon_mappe) if d.startswith("time_minutt=")])

print("\nSimulerer datastrømming over 10-minutters intervaller...")
for intervall_dir in alle_intervaller:
    # Hent time og minutt fra mappenavnet
    match = re.search(r"time_minutt=(\d{2})_(\d{2})", intervall_dir)
    if match:
        time, minutt = match.groups()
        
        intervall_sti = os.path.join(partisjon_mappe, intervall_dir)
        print(f"\nProsesserer data for {time}:{minutt}...")
        
        # Finn alle geoparquet-filer for dette intervallet
        filer = glob.glob(os.path.join(intervall_sti, "**/*.parquet"), recursive=True)
        
        # Les og bearbeid hver fil
        intervall_data = []
        for fil in filer:
            gdf = gpd.read_parquet(fil)
            intervall_data.append(gdf)
            
        if intervall_data:
            samlet_data = pd.concat(intervall_data)
            print(f"Kl. {time}:{minutt}: Lastet {len(samlet_data)} rader fra {len(filer)} filer")
            
            # Her kan du gjøre din analyse for dette intervallet
            print(f"  Gjennomsnittlig hastighet (SOG): {samlet_data['speed_over_ground'].mean():.2f}")
            print(f"  Antall unike skip: {samlet_data['mmsi'].nunique()}")
            
            # Sorter data etter nøyaktig tidspunkt innen intervallet
            samlet_data_sortert = samlet_data.sort_values('date_time_utc')
            
            # Her kunne du lagt til mer analyse eller visualisering
            # For eksempel se på hvordan skip beveger seg innen dette 10-minutters intervallet
            
        else:
            print(f"Ingen data funnet for kl. {time}:{minutt}")
        
        # Simuler tid mellom intervaller (kortere siden dette er en mer detaljert simulering)
        time_module.sleep(0.5)  # Vent et halvt sekund mellom hvert intervall
        
print("\nDatastrømming komplett!")


Simulerer datastrømming over 10-minutters intervaller...

Datastrømming komplett!


## Main

Mappen Data finnes allerede


OPPSUMMERING
Katalog: C:\Users\ANDRI\Documents\Bachelor\Programmering\kaidata_geolake\Data
Mappestruktur: Raw=C:\Users\ANDRI\Documents\Bachelor\Programmering\kaidata_geolake\Data\raw, Processed=C:\Users\ANDRI\Documents\Bachelor\Programmering\kaidata_geolake\Data\processed
Totalt antall geografiske filer funnet: 0
Antall parquet-filer som allerede var GeoParquet: 0
Antall filer konvertert til GeoParquet: 0
Antall filer som ikke kunne konverteres: 0

Totalt konverterte filer: 0
Filer som allerede var i GeoParquet-format: 0


In [39]:
# ## Konvertere én enkelt fil - eksempel

# Definerer stier basert på den opprettede mappestrukturen
if os.path.exists(ROT_FOLDER):
    rå_mappe = os.path.join(ROT_FOLDER, "raw")
    prosessert_mappe = os.path.join(ROT_FOLDER, "processed")

    # Eksempelfil - dette er for demonstrasjon,
    eksempelfil = os.path.join(rå_mappe, "veier_kristiansand.geojson")

    # Utfør kun hvis filen eksisterer (endre denne kommentaren hvis du har en faktisk fil)
    if os.path.exists(eksempelfil):
        print(f"Konverterer {eksempelfil}...")
        resultat = converter_parquet(eksempelfil)
        if resultat:
            print(f"Filen ble konvertert og lagret til: {resultat}")

            # Lese og vise data
            try:
                gdf = gpd.read_parquet(resultat)
                display(gdf.head())

                # Enkel kartvisning
                import matplotlib.pyplot as plt
                fig, ax = plt.subplots(figsize=(10,8))
                gdf.plot(ax=ax)
                plt.title("Konvertert GeoParquet-fil")
                plt.show()
            except Exception as e:
                print(f"Kunne ikke vise data: {e}")
    else:
        print(f"Ingen eksempelfil funnet på {eksempelfil}")
        print("For å teste konvertering:")
        print(f"1. Legg en geografisk fil (f.eks. GeoJSON eller Shapefile) i {rå_mappe}")
        print(f"2. Kjør denne cellen på nytt med korrekt filsti")
else:
    print(f"Rotmappe {ROT_FOLDER} finnes ikke. Kjør seksjon 8 først.")

Ingen eksempelfil funnet på ./data\raw\veier_kristiansand.geojson
For å teste konvertering:
1. Legg en geografisk fil (f.eks. GeoJSON eller Shapefile) i ./data\raw
2. Kjør denne cellen på nytt med korrekt filsti
